# WSL Ubuntu에서 Triton vs PyTorch 비교 (Wine 데이터셋)

## 목표
- PyTorch 기본 학습 vs Triton 커널 직접 제어
- CPU 환경에서 병렬 처리 구조 이해
- Wine 데이터셋으로 실전 비교

## 1. 환경 설정

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import numpy as np
from dataset_wine import get_wine_loaders, set_seed

# Triton import
try:
    import triton
    import triton.language as tl
    TRITON_AVAILABLE = True
    print(f"✅ Triton 버전: {triton.__version__}")
except ImportError:
    TRITON_AVAILABLE = False
    print("⚠️ Triton 미설치. 설치: pip install triton")

print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

set_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

✅ Triton 버전: 3.5.1
PyTorch: 2.9.0+cpu
Device: CPU


In [14]:
# 데이터 로드
train_loader, val_loader, test_loader, input_dim, num_classes = get_wine_loaders(batch_size=64)
print(f"입력 차원: {input_dim}, 클래스 수: {num_classes}")

입력 차원: 13, 클래스 수: 3


## 2. 모델 및 함수 정의

In [15]:
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes, h1=64, h2=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1), nn.ReLU(),
            nn.Linear(h1, h2), nn.ReLU(),
            nn.Linear(h2, num_classes)
        )
    
    def forward(self, x):
        return self.net(x)

def accuracy(model, loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            pred = model(x).argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total

## 3. 사진 전: PyTorch 기본 학습 (L2)

In [16]:
def train_pytorch(model, epochs=50, lr=1e-3, weight_decay=1e-4):
    """PyTorch 기본 학습 (L2 정규화)"""
    opt = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce = nn.CrossEntropyLoss()
    best_val = 0.0
    
    start = time.time()
    for ep in range(1, epochs+1):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            loss = ce(model(x), y)
            loss.backward()
            opt.step()
        
        val_acc = accuracy(model, val_loader)
        best_val = max(best_val, val_acc)
        if ep % 10 == 0:
            print(f"Epoch {ep}: val_acc={val_acc:.4f}")
    
    train_time = time.time() - start
    test_acc = accuracy(model, test_loader)
    return best_val, test_acc, train_time

model_pt = MLP(input_dim, num_classes).to(device)
val_pt, test_pt, time_pt = train_pytorch(model_pt)

print(f"\n✅ PyTorch 결과: Val={val_pt:.4f}, Test={test_pt:.4f}, Time={time_pt:.2f}s")

Epoch 10: val_acc=0.9722
Epoch 20: val_acc=0.9722
Epoch 30: val_acc=0.9722
Epoch 40: val_acc=0.9722
Epoch 50: val_acc=0.9722

✅ PyTorch 결과: Val=0.9722, Test=0.9722, Time=0.30s


## 4. 사진 후: Triton 커널 구현

In [17]:
if TRITON_AVAILABLE:
    @triton.jit
    def add_kernel(a_ptr, b_ptr, c_ptr, n, BLOCK_SIZE: tl.constexpr):
        """간단한 벡터 덧셈 커널 (예시)"""
        pid = tl.program_id(0)
        offs = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
        mask = offs < n
        a = tl.load(a_ptr + offs, mask=mask)
        b = tl.load(b_ptr + offs, mask=mask)
        tl.store(c_ptr + offs, a + b, mask=mask)
    
    def triton_add(a, b):
        c = torch.empty_like(a)
        n = a.numel()
        grid = lambda meta: (triton.cdiv(n, meta['BLOCK_SIZE']),)
        add_kernel[grid](a, b, c, n, BLOCK_SIZE=256)
        return c
    
    # 테스트
    test_a = torch.arange(1000, dtype=torch.float32)
    test_b = torch.arange(1000, dtype=torch.float32)
    
    pt_result = test_a + test_b
    tr_result = triton_add(test_a, test_b)
    
    print(f"\n✅ Triton 커널 테스트:")
    print(f"  정확도 일치: {torch.allclose(pt_result, tr_result)}")
    print(f"  최대 오차: {torch.max(torch.abs(pt_result - tr_result)).item():.2e}")
    
    # 성능 비교
    N = 1_000_000
    a = torch.randn(N)
    b = torch.randn(N)
    
    start = time.time()
    _ = a + b
    pt_time = time.time() - start
    
    _ = triton_add(a, b)  # warm-up
    start = time.time()
    _ = triton_add(a, b)
    tr_time = time.time() - start
    
    print(f"\n⏱️ 성능 비교 (N={N:,}):")
    print(f"  PyTorch: {pt_time:.6f}s")
    print(f"  Triton:  {tr_time:.6f}s")
    print(f"  비율: {tr_time/pt_time:.2f}x")
else:
    print("⚠️ Triton이 설치되지 않아 건너뜁니다.")

RuntimeError: 0 active drivers ([]). There should only be one.

## 5. 시각화 및 비교

In [ ]:
if TRITON_AVAILABLE:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # 모델 정확도
    ax1.bar(['PyTorch'], [test_pt], alpha=0.7, label='Test Acc')
    ax1.set_ylabel('정확도')
    ax1.set_title('모델 성능')
    ax1.set_ylim(0, 1)
    ax1.legend()
    
    # 커널 성능
    ax2.bar(['PyTorch', 'Triton'], [pt_time, tr_time], alpha=0.7)
    ax2.set_ylabel('시간 (초)')
    ax2.set_title('벡터 덧셈 성능')
    
    plt.tight_layout()
    plt.show()
    
    print("\n" + "="*60)
    print("핵심 학습 포인트")
    print("="*60)
    print("\n1. PyTorch (사진 전):")
    print("   - 매우 편리한 API")
    print("   - 내부 동작이 블랙박스")
    print("   - 병렬 처리 구조를 알 수 없음")
    print("\n2. Triton (사진 후):")
    print("   - 커널을 직접 작성 (학습 필요)")
    print("   - program_id, BLOCK_SIZE로 병렬 구조 명시")
    print("   - tl.load/store로 메모리 접근 제어")
    print("   - GPU 프로그래밍 개념 학습")
    print("\n3. CPU Fallback:")
    print("   - GPU 없이도 커널 프로그래밍 학습 가능")
    print("   - 성능은 PyTorch가 더 빠를 수 있음 (BLAS)")
    print("   - 목적은 성능 < 병렬 구조 이해")
    print("\n4. WSL Ubuntu:")
    print("   - Triton 설치 안정적")
    print("   - Linux 딥러닝 생태계 호환")
    print("   - 향후 GPU 전환 용이")
    print("="*60)

## 6. 결론

### Before (PyTorch)
```python
y = a + b  # ❓ 어떻게 동작할까?
```

### After (Triton)
```python
@triton.jit
def add_kernel(...):
    pid = tl.program_id(0)  # ✅ 블록 ID
    offs = pid * BLOCK_SIZE  # ✅ 시작 위치
    # ✅ 명시적 메모리 제어
```

### 핵심 메시지

> **"GPU 없이도 WSL Ubuntu에서 Triton으로**  
> **CPU 커널을 직접 제어하며 병렬 구조를 학습할 수 있다!"**